In [1]:
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import xgboost
import sklearn
debug_mode = True
gc.collect()

0

# load data

In [2]:
train = pd.read_feather('../data/2021玉山人工智慧公開挑戰賽冬季賽訓練資料集.feather')
if debug_mode == True:
    train = train.sample(int(len(train)*0.33))
test = pd.read_feather('../data/需預測的顧客名單及提交檔案範例.feather')
print(train.shape,test.shape)

(10881965, 53) (500000, 4)


# select data by chid

In [3]:
train = train.loc[train.chid.isin(test.chid.unique())]
df = train.append(test).reset_index(drop=True)
print(df.shape,train.shape,test.shape)
display(df)

(11381964, 56) (10881964, 53) (500000, 4)


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,naty,poscd,cuorg,slam,gender_code,age,primary_card,top1,top2,top3
0,20.0,10379793.0,37,1.0,4380.700782,1.0,0.0,0.0,0.0,1.00,...,1.0,99.0,30.0,102382.569589,0.0,5.0,1.0,NaN,NaN,NaN
1,22.0,10124560.0,other,1.0,20298.249036,1.0,0.0,0.0,0.0,1.00,...,1.0,99.0,30.0,123763.112816,0.0,5.0,1.0,NaN,NaN,NaN
2,19.0,10445673.0,37,1.0,558.394631,1.0,0.0,0.0,0.0,1.00,...,1.0,99.0,30.0,79291.240239,1.0,4.0,1.0,NaN,NaN,NaN
3,13.0,10484363.0,25,1.0,10651.037325,1.0,0.0,0.0,0.0,1.00,...,1.0,99.0,30.0,NaN,1.0,3.0,0.0,NaN,NaN,NaN
4,10.0,10172273.0,37,2.0,5357.428338,1.0,1.0,0.0,0.0,0.38,...,1.0,6.0,30.0,92480.560013,1.0,3.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11381959,NaN,10433239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.0,48.0,25.0
11381960,NaN,10235119.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,36.0,12.0
11381961,NaN,10267595.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0,26.0,15.0
11381962,NaN,10160606.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.0,15.0,13.0


In [4]:
not_feature = ['index','chid','shop_tag']
origin_features = list(set(df.columns.tolist())-set(not_feature))
len(origin_features)

54

# 處理缺失值

In [5]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
miss_table = missing_values_table(df)
miss_table

Your selected dataframe has 56 columns.
There are 55 columns that have missing values.


,Missing Values,% of Total Values
top3,10881964,95.6
top2,10881964,95.6
top1,10881964,95.6
slam,1078013,9.5
age,638993,5.6
gender_code,638993,5.6
trdtp,500137,4.4
educd,500004,4.4
naty,500002,4.4
masts,500002,4.4


In [6]:
train_miss = missing_values_table(train)
print(train.shape)
train_miss

Your selected dataframe has 53 columns.
There are 9 columns that have missing values.
(10881964, 53)


,Missing Values,% of Total Values
slam,578013,5.3
gender_code,138993,1.3
age,138993,1.3
trdtp,137,0.0
educd,4,0.0
masts,2,0.0
naty,2,0.0
poscd,2,0.0
cuorg,2,0.0


In [7]:
test_miss = missing_values_table(test)
print(test.shape)
test_miss

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.
(500000, 4)


,Missing Values,% of Total Values


In [8]:
miss_table.loc[set(train_miss.index)-set(test_miss.index)]

,Missing Values,% of Total Values
masts,500002,4.4
poscd,500002,4.4
naty,500002,4.4
gender_code,638993,5.6
trdtp,500137,4.4
cuorg,500002,4.4
age,638993,5.6
educd,500004,4.4
slam,1078013,9.5


In [9]:
row_drop_columns = list(set(train_miss.index)-set(test_miss.index))
row_drop_columns

['masts',
 'poscd',
 'naty',
 'gender_code',
 'trdtp',
 'cuorg',
 'age',
 'educd',
 'slam']

In [10]:
# row axis drop
train = train.dropna(subset=row_drop_columns,axis=0)
gc.collect()

123

In [11]:
train_miss = missing_values_table(train)
print(train.shape)
train_miss

Your selected dataframe has 53 columns.
There are 0 columns that have missing values.
(10184090, 53)


,Missing Values,% of Total Values


In [12]:
test_miss = missing_values_table(test)
print(test.shape)
test_miss

Your selected dataframe has 4 columns.
There are 0 columns that have missing values.
(500000, 4)


,Missing Values,% of Total Values


# 

# final check data

In [13]:
train = train.reset_index(drop=True)
print(train.shape)
print(train.isnull().sum().sum())
train.head()

(10184090, 53)
0


,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,...,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,20.0,10379793.0,37,1.0,4380.700782,1.0,0.0,0.0,0.0,1.00,...,2.0,3.0,15.0,1.0,99.0,30.0,102382.569589,0.0,5.0,1.0
1,22.0,10124560.0,other,1.0,20298.249036,1.0,0.0,0.0,0.0,1.00,...,1.0,2.0,11.0,1.0,99.0,30.0,123763.112816,0.0,5.0,1.0
2,19.0,10445673.0,37,1.0,558.394631,1.0,0.0,0.0,0.0,1.00,...,1.0,5.0,15.0,1.0,99.0,30.0,79291.240239,1.0,4.0,1.0
3,10.0,10172273.0,37,2.0,5357.428338,1.0,1.0,0.0,0.0,0.38,...,1.0,3.0,27.0,1.0,6.0,30.0,92480.560013,1.0,3.0,1.0
4,12.0,10296216.0,15,1.0,6159.357893,0.0,0.0,1.0,0.0,0.00,...,1.0,2.0,12.0,1.0,8.0,30.0,128396.971578,1.0,5.0,1.0


In [14]:
test = test.reset_index(drop=True)
print(test.shape)
print(test.isnull().sum().sum())
test.head()

(500000, 4)
0


,chid,top1,top2,top3
0,10128239,18,10,6
1,10077943,48,22,6
2,10277876,22,10,39
3,10364842,37,19,13
4,10392717,2,36,26


# save data

In [15]:
train.to_feather("../data/train_handle_nan.feather")
test.to_feather("../data/test_handle_nan.feather")
print('save done!')

save done!
